Setup

In [37]:
# imports
from __future__ import print_function
import stardog  # querying the graph
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import SVG, display
import svgwrite as sw
import time
import ipywidgets as widgets
import pandas as pd
from datetime import datetime

# connection details for Stardog
conn_details = {
  'endpoint': 'http://localhost:5820',
  'username': 'admin',
  'password': 'admin'
}

Query for spaces and their boundaries

In [38]:
with stardog.Connection('ldachackathon', **conn_details) as conn:
  results = conn.select("""
PREFIX bot: <https://w3id.org/bot#>
PREFIX sosa: <http://www.w3.org/ns/sosa/>
PREFIX ssn: <http://www.w3.org/ns/ssn/systems/>
PREFIX props: <https://w3id.org/props#>
select ?space ?boundary WHERE { 
  ?space a bot:Space ;
    props:spaceBoundary ?boundary
}""")

def res_to_obj(res):
  return { 'boundary': res['boundary']['value'], 'space': res['space']['value'] }
    
mapped = map(res_to_obj, results['results']['bindings'])
mapped = list(mapped)

In [39]:
sensor_types = ['http://elite.polito.it/ontologies/dogont.owl#HumiditySensor',
'http://elite.polito.it/ontologies/dogont.owl#LightSensor',
'http://elite.polito.it/ontologies/dogont.owl#ThermostatTemperatureSensor',
'http://elite.polito.it/ontologies/dogont.owl#SingleTemperatureSensor',
'http://elite.polito.it/ontologies/dogont.owl#TemperatureSensor']

In [45]:
# Parse the result
def parse(result):
    return [
            tuple(float(x) for x in r.strip().split(' '))
            for r in result[9:-2].split(',')
           ]

rooms = [{"points": parse(i['boundary'])} for i in mapped ]

# Define constants for drawing
SHEET_SIZE = 500
DRAW_SIZE = 300
offset = int((SHEET_SIZE - DRAW_SIZE) / 2)
points = []
for room in rooms:
    points += room["points"]
xpts, ypts = zip(*points)

min_x = min(xpts)
min_y = min(ypts)
range_x = max(xpts) - min_x
range_y = max(ypts) - min_y

# Normalise polygon dimensions to get in draw_size
# All values must be positives
def get_path(points, min_x, min_y, range_x, range_y):
    return [
        (
            (p[0] - min_x) / range_x * DRAW_SIZE,
            (p[1] - min_y) / range_y * DRAW_SIZE
        )
        for p in points
    ]

# Add offset to get picture in middle of sheet
def add_offset(points, x_offset, y_offset):
    return [
        (
            p[0] + x_offset,
            p[1] + y_offset
        )
        for p in points
    ]

class RoomVisualizer:
    def __init__(self):
        pass
    
    def set_sensor_type(self, type):
        self.sensor_type = type
        
    def set_time(self, time):
        self.time = time
        
    def draw_all_rooms(self):
        # Dummy coloring
        if self.sensor_type == sensor_types[0]:
            fill = "green"
        else:
            fill = "red"
        # TODO: Query sensors of given type and retrieve their values, connected to a space
        drw_params = {'fill': fill, 'fill-opacity': 1}
        drw = sw.Drawing('test.svg',size=(SHEET_SIZE, SHEET_SIZE))

        for room in rooms:
            path = get_path(room['points'], min_x, min_y, range_x, range_y)
            path = add_offset(path, offset, offset)
            drw.add(drw.polygon(path, stroke="black", **drw_params))

        display(SVG(drw.tostring()))

visualizer = RoomVisualizer()

start_date = datetime(2018, 4, 24)
end_date = datetime(2018, 5, 24)

dates = pd.date_range(start_date, end_date, freq='D')

options = [(date.strftime(' %d %b %Y '), date) for date in dates]
index = 0

selection_range_slider = widgets.SelectionSlider(
    options=options,
    index=index,
    description='Time',
    orientation='horizontal',
    layout={'width': '500px'}
)

selection_range_slider

@interact(sensor_type=sensor_types, time=selection_range_slider)
def set_parameters(sensor_type, time):
    visualizer.set_sensor_type(sensor_type)
    visualizer.set_time(time)
    visualizer.draw_all_rooms()

interactive(children=(Dropdown(description='sensor_type', options=('http://elite.polito.it/ontologies/dogont.o…